# 多项式回归实现与应用

In [ ]:
# 加载示例数据
x = [4, 8, 12, 25, 32, 43, 58, 63, 69, 79]
y = [20, 33, 50, 56, 42, 31, 33, 46, 65, 75]

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

plt.scatter(x, y)

## 2 次多项式拟合

In [ ]:
# 通过手动指定多项式阶数 m 的大小，就只需要确定多项式系数 w 的值
def func(p, x):
    # 根据公式，定义 2 次多项式函数
    w0, w1, w2 = p
    f = w0 + w1 * x + w2 * x * x
    return f


def err_func(p, x, y):
    # 残差函数（观测值与拟合值之间的差距）
    ret = func(p, x) - y
    return ret

In [ ]:
# 使用最小二乘法求解最优参数的过程
import numpy as np
from scipy.optimize import leastsq

p_init = np.random.randn(3)  # 生成 3 个随机数
# 使用 Scipy 提供的最小二乘法函数得到最佳拟合参数
parameters = leastsq(err_func, p_init, args=(np.array(x), np.array(y)))

print("Fitting Parameters: ", parameters[0])

In [ ]:
# 绘制拟合图像时需要的临时点
x_temp = np.linspace(0, 80, 10000)

# 绘制拟合函数曲线
plt.plot(x_temp, func(parameters[0], x_temp), "r")

# 绘制原数据点
plt.scatter(x, y)

## N 次多项式拟合

In [ ]:
def fit_func(p, x):
    """根据公式，定义 n 次多项式函数"""
    f = np.poly1d(p)
    return f(x)


def err_func(p, x, y):
    """残差函数（观测值与拟合值之间的差距）"""
    ret = fit_func(p, x) - y
    return ret


def n_poly(n):
    """n 次多项式拟合"""
    p_init = np.random.randn(n)  # 生成 n 个随机数
    parameters = leastsq(err_func, p_init, args=(np.array(x), np.array(y)))
    return parameters[0]

n_poly(3)

In [ ]:
# 绘制拟合图像时需要的临时点
x_temp = np.linspace(0, 80, 10000)

# 绘制子图
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(x_temp, fit_func(n_poly(4), x_temp), "r")
axes[0, 0].scatter(x, y)
axes[0, 0].set_title("m = 3")

axes[0, 1].plot(x_temp, fit_func(n_poly(5), x_temp), "r")
axes[0, 1].scatter(x, y)
axes[0, 1].set_title("m = 4")

axes[0, 2].plot(x_temp, fit_func(n_poly(6), x_temp), "r")
axes[0, 2].scatter(x, y)
axes[0, 2].set_title("m = 5")

axes[1, 0].plot(x_temp, fit_func(n_poly(7), x_temp), "r")
axes[1, 0].scatter(x, y)
axes[1, 0].set_title("m = 6")

axes[1, 1].plot(x_temp, fit_func(n_poly(8), x_temp), "r")
axes[1, 1].scatter(x, y)
axes[1, 1].set_title("m = 7")

axes[1, 2].plot(x_temp, fit_func(n_poly(9), x_temp), "r")
axes[1, 2].scatter(x, y)
axes[1, 2].set_title("m = 8")


print('过拟和 Overfitting 随着 m 次数的增加曲线呈现出明显的震荡')

### 用 scikit-learn 进行多项式拟合

- 类 sklearn.preprocessing.PolynomialFeatures(degree=2, interaction_only=False, include_bias=True) 主要作用是产生多项式特征矩阵
  - degree: 多项式次数，默认为 2 次多项式
  - interaction_only: 默认为 False，为 True 则产生相互影响的特征集
  - include_bias: 默认为 True，包含多项式中的截距项
- 多项式回归相当于线性回归的特殊形式，相当于一元高次多项式到多元一次多项式之间的转换
- 当多项式为一元高次或者多元高次时，特征矩阵的表达和计算过程变得比较复杂

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

X = [2, -1, 3]
X_reshape = np.array(X).reshape(len(X), 1)  # 转换为列向量
# 使用 PolynomialFeatures 自动生成特征矩阵
PolynomialFeatures(degree=2, include_bias=False).fit_transform(X_reshape)

In [ ]:
x = np.array(x).reshape(len(x), 1)  # 转换为列向量
y = np.array(y).reshape(len(y), 1)

# 用 sklearn 得到 2 次多项式回归特征矩阵
poly_features = PolynomialFeatures(degree=2, include_bias=False)
poly_x = poly_features.fit_transform(x)

poly_x

In [ ]:
from sklearn.linear_model import LinearRegression

# 定义线性回归模型
model = LinearRegression()
model.fit(poly_x, y)  # 训练

# 得到模型拟合参数
model.intercept_, model.coef_

In [ ]:
# 绘制拟合图像
x_temp = np.array(x_temp).reshape(len(x_temp), 1)
poly_x_temp = poly_features.fit_transform(x_temp)

plt.plot(x_temp, model.predict(poly_x_temp), "r")
plt.scatter(x, y)

## 多项式回归预测

In [ ]:
# 导入并预览「世界麻疹疫苗接种率」数据集
import pandas as pd

# 加载数据集 
df = pd.read_csv(
    # "https://cdn.aibydoing.com/hands-on-ai/files/course-6-vaccine.csv",
    "../../data/course-6-vaccine.csv",
    header=0,
)
df

In [ ]:
# 定义 x, y 的取值
x = df["Year"]
y = df["Values"]
# 绘图
plt.plot(x, y, "r")
plt.scatter(x, y)

In [ ]:
# 划分 dateframe 为训练集和测试集
train_df = df[: int(len(df) * 0.7)]
test_df = df[int(len(df) * 0.7) :]

# 定义训练和测试使用的自变量和因变量
X_train = train_df["Year"].values
y_train = train_df["Values"].values

X_test = test_df["Year"].values
y_test = test_df["Values"].values

In [ ]:
# 建立线性回归模型
model = LinearRegression()
model.fit(X_train.reshape(len(X_train), 1), y_train.reshape(len(y_train), 1))
results = model.predict(X_test.reshape(len(X_test), 1))
results  # 线性回归模型在测试集上的预测结果

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

print("线性回归平均绝对误差: ", mean_absolute_error(y_test, results.flatten()))
print("线性回归均方误差: ", mean_squared_error(y_test, results.flatten()))

In [ ]:
# 2 次多项式回归特征矩阵
poly_features_2 = PolynomialFeatures(degree=2, include_bias=False)
poly_X_train_2 = poly_features_2.fit_transform(X_train.reshape(len(X_train), 1))
poly_X_test_2 = poly_features_2.fit_transform(X_test.reshape(len(X_test), 1))

# 2 次多项式回归模型训练与预测
model = LinearRegression()
model.fit(poly_X_train_2, y_train.reshape(len(X_train), 1))  # 训练模型

results_2 = model.predict(poly_X_test_2)  # 预测结果

results_2.flatten()  # 打印扁平化后的预测结果

In [ ]:
print("2 次多项式回归平均绝对误差: ", mean_absolute_error(y_test, results_2.flatten()))
print("2 次多项式均方误差: ", mean_squared_error(y_test, results_2.flatten()))

In [ ]:
# 更高次多项式回归预测
from sklearn.pipeline import make_pipeline

X_train = X_train.reshape(len(X_train), 1)
X_test = X_test.reshape(len(X_test), 1)
y_train = y_train.reshape(len(y_train), 1)

for m in [3, 4, 5]:
    model = make_pipeline(PolynomialFeatures(m, include_bias=False), LinearRegression())
    model.fit(X_train, y_train)
    pre_y = model.predict(X_test)
    print("{} 次多项式回归平均绝对误差: ".format(m), mean_absolute_error(y_test, pre_y.flatten()))
    print("{} 次多项式均方误差: ".format(m), mean_squared_error(y_test, pre_y.flatten()))
    print("---")

### 多项式回归预测次数选择

- 选择一个误差指标，例如选择 MSE，计算出该指标随多项式次数增加而变化的图像

In [ ]:
# 计算 m 次多项式回归预测结果的 MSE 评价指标并绘图
mse = []  # 用于存储各最高次多项式 MSE 值
m = 1  # 初始 m 值
m_max = 10  # 设定最高次数
while m <= m_max:
    model = make_pipeline(PolynomialFeatures(m, include_bias=False), LinearRegression())
    model.fit(X_train, y_train)  # 训练模型
    pre_y = model.predict(X_test)  # 测试模型
    mse.append(mean_squared_error(y_test, pre_y.flatten()))  # 计算 MSE
    m = m + 1

print("MSE 计算结果: ", mse)
# 绘图
plt.plot([i for i in range(1, m_max + 1)], mse, "r")
plt.scatter([i for i in range(1, m_max + 1)], mse)

# 绘制图名称等
plt.title("MSE of m degree of polynomial regression")
plt.xlabel("m")
plt.ylabel("MSE")

考虑到模型的泛化能力，避免出现过拟合，这里就可以选择 3 次多项式为最优回归预测模型

## 比特币价格预测及绘图

In [ ]:
import pandas as pd

df = None
df = pd.read_csv(
    # "https://cdn.aibydoing.com/hands-on-ai/files/challenge-2-bitcoin.csv",
    "../../data/challenge-2-bitcoin.csv",
    header=0,
)
df.head()

In [ ]:
data = df [["btc_market_price", "btc_total_bitcoins","btc_transaction_fees"]]
data.head()

In [ ]:
# 将 3 列数据，分别绘制在横向排列的 3 张子图中
## 需设置各图横纵轴名称，横轴统一为 time，纵轴为各自列名称
from matplotlib import pyplot as plt
%matplotlib inline

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
axes[0].plot(data['btc_market_price'], 'green')
axes[0].set_xlabel('time')
axes[0].set_ylabel('btc_market_price')

axes[1].plot(data['btc_total_bitcoins'], 'blue')
axes[1].set_xlabel('time')
axes[1].set_ylabel('btc_total_bitcoins')

axes[2].plot(data['btc_transaction_fees'], 'brown')
axes[2].set_xlabel('time')
axes[2].set_ylabel('btc_transaction_fees')

In [ ]:
# 数据集
# 特征（Features）是「比特币总量」和「比特币交易费用」
# 目标值为「比特币市场价格」
def split_dataset():
    """
    参数:
    无

    返回:
    X_train, y_train, X_test, y_test -- 训练集特征、训练集目标、测试集特征、测试集目标
    """
    train_data = data[: int(len(data) * 0.7)]
    test_data = data[int(len(data) * 0.7):]
    X_train = train_data[["btc_total_bitcoins", 'btc_transaction_fees']]
    X_test = test_data[["btc_total_bitcoins", 'btc_transaction_fees']]
    y_train = train_data[["btc_market_price"]]
    y_test = test_data[["btc_market_price"]]

    return X_train, y_train, X_test, y_test


len(split_dataset()[0]), len(split_dataset()[1]), len(split_dataset()[2]), len(split_dataset()[
    3]), split_dataset()[0].shape, split_dataset()[1].shape, split_dataset()[2].shape, split_dataset()[3].shape

### 3 次多项式回归预测挑战

- 使用 scikit-learn 构建 3 次多项式回归预测模型，并计算预测结果的 MAE 评价指标，同时作为 poly3() 函数返回值

In [ ]:
# 加载必要模块
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# 加载数据
X_train = split_dataset()[0]
y_train = split_dataset()[1]
X_test = split_dataset()[2]
y_test = split_dataset()[3]


def poly3():
    """
    参数:
    无

    返回:
    mae -- 预测结果的 MAE 评价指标
    """
    poly_features = PolynomialFeatures(degree=3, include_bias=False)
    poly_X_train = poly_features.fit_transform(X_train)
    poly_X_test = poly_features.fit_transform(X_test)

    model = LinearRegression()
    model.fit(poly_X_train, y_train)
    pre_y = model.predict(poly_X_test)

    mae = mean_absolute_error(y_test, pre_y.flatten())

    return mae


poly3()

In [ ]:
# N 次多项式回归预测绘图 计算 1,2,…,10 次多项式回归预测结果的 MSE 评价指标
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error


def poly_plot(N):
    """
    参数:
    N -- 标量, 多项式次数

    返回:
    mse -- N 次多项式预测结果的 MSE 评价指标列表
    """

    m = 1
    mse = []
    while m <= N:
        model = make_pipeline(PolynomialFeatures(
            m, include_bias=False), LinearRegression())
        model.fit(X_train, y_train)  # 训练模型
        pre_y = model.predict(X_test)  # 测试模型
        mse.append(mean_squared_error(y_test, pre_y.flatten()))  # 计算 MSE
        m = m + 1

    return mse


poly_plot(10)[:10:3]

In [ ]:
# 将 MSE 评价指标绘制成线型图 规定：将 poly_plot(10) 返回的 MSE 列表绘制成组合图（线形图+散点图）。线型图为红色
mse = poly_plot(10)

plt.plot([i for i in range(1, 11)], mse, "r")
plt.scatter([i for i in range(1, 11)], mse)

plt.title("MSE of m degree of polynomial regression")
plt.xlabel("N")
plt.ylabel("MSE")